In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from utils import config
from utils import utils
from datasets import mhad_dataset
# from dataset import real_fake_data
# from dataset import ntu_data
from utils.modules import Regressor, FeatureExtractor, ActivityClassifier

In [2]:
#region MHAD dataset
accel_list, skel_list, label_list = [], [], []

for s in config.MHAD_TRAIN_S_IDS:
    s_dir = os.path.join(config.mhad_data_dir, s)

    files = sorted(os.listdir(s_dir))
    # group files by 4 a*_s*_t*{intertial_std.npy, inertial.npy, skeleton_upsample_normal.npy, skeleton.npy}
    file_groups = [files[i:i+4] for i in range(0, len(files), 4)]
    for group in file_groups:
        '''
        group = ['a?_s?_t?_inertial.npy', 'a?_s?_t?_inertial_std.npy', 'a?_s?_t?_skeleton.npy', 'a?_s?_t?_skeleton_upsample.npy']
        '''
        label = group[0].split('_')[0]
        if label not in config.mhad_actions:
            continue

        accel_file = group[1] # 'a?_s?_t?_inertial_std.npy'
        accel = np.load(os.path.join(s_dir, accel_file)) # (3, N)

        skel_file = group[3] # 'a?_s?_t?_skeleton_upsample.npy'
        skel = np.load(os.path.join(s_dir, skel_file)) # (3, joints, N)
        # right hand: shoulder, elbow, wrist
        skel = skel[:, [8, 9, 10], :] # (3, 3, N)


        accel_list.append(accel)
        skel_list.append(skel)
        label_list.append(label)  

mhad_dataset = mhad_dataset.MHADDataset(accel_list, skel_list, label_list)
mhad_dataloader = DataLoader(mhad_dataset, batch_size=config.batch_size, shuffle=True)
#endregion

In [3]:
# region Models
window = int(config.mhad_window_sec * config.mhad_accel_sampling_rate)
pose2imu_model = Regressor(
    in_ch=config.in_ch,
    num_joints=config.num_joints,
    window_length=window
).to(config.device, non_blocking=True)

fe_model = FeatureExtractor(window_size=window).to(
    config.device, non_blocking=True)
ac_model = ActivityClassifier(f_in=config.ac_fin, n_classes=len(
    config.mhad_actions)).to(config.device, non_blocking=True)

# >>> Loss + Optimization <<< #
MSELoss = nn.MSELoss()


def cosine_similarity_loss(output, target):
    cosine_loss = 1 - F.cosine_similarity(output, target, dim=1)
    return cosine_loss.mean()


CrossEntropyLoss = nn.CrossEntropyLoss()

params = (
    list(pose2imu_model.parameters())
    + list(fe_model.parameters())
    + list(ac_model.parameters())
)
optimizer = torch.optim.Adam(params, lr=config.lr)  # , weight_decay=1e-03)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=10
)
# endregion

In [4]:
#region Train
train_loss_history, train_f1_history, train_accuracy_history = [], [], []
val_loss_history, val_f1_history, val_accuracy_history = [], [], []

best_pose2imu_model_state, best_fe_model_state, best_ac_model_state = None, None, None
best_val_f1 = 0
best_epoch = float('inf')

epochs_no_improve = 0
log = ''

for epoch in range(config.epochs):

    # Initialize tracking variables
    total_train_loss, total_train_mse_loss, total_train_similarity_loss, total_train_activity_loss = 0, 0, 0, 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    # Set models to training mode
    pose2imu_model.train()
    fe_model.train()
    ac_model.train()

    for accel, skel, label in mhad_dataloader:
        # move to GPU
        accel = accel.to(config.device, non_blocking=True)
        skel = skel.to(config.device, non_blocking=True)
        label = label.to(config.device, non_blocking=True)

        # -- Forward pass
        sim_accel = pose2imu_model(skel) # Problem
        mse_loss = MSELoss(sim_accel, accel)

        accel_f = fe_model(accel)
        sim_accel_f = fe_model(sim_accel)
        similarity_loss = cosine_similarity_loss(sim_accel_f, accel_f)

        # Combine logits for efficiency
        comb_f = torch.cat((accel_f, sim_accel_f), dim=0)
        comb_label = torch.cat((label, label), dim=0)
        logits = ac_model(comb_f)
        activity_loss = CrossEntropyLoss(logits, comb_label)

        # Total loss
        total_loss = mse_loss + config.scenario2_alpha * activity_loss + config.scenario2_beta * similarity_loss

        # -- Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Accumulate losses
        total_train_loss += total_loss.item()
        total_train_mse_loss += mse_loss.item()
        total_train_similarity_loss += similarity_loss.item()
        total_train_activity_loss += activity_loss.item()

        # Compute F1 and Accuracy on the batch
        pred_labels = torch.argmax(logits[:label.size(0)], dim=1)  # Only consider real logits for metrics
        all_pred_labels.extend(pred_labels.cpu().numpy())
        all_true_labels.extend(label.cpu().numpy())

        correct_predictions += (pred_labels == label).sum().item()
        total_predictions += label.size(0)

    # Average losses
    average_train_loss = total_train_loss / len(mhad_dataloader)
    train_loss_history.append(average_train_loss)

    # F1 Score
    train_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    train_f1_history.append(train_f1)

    # Accuracy
    train_accuracy = correct_predictions / total_predictions
    train_accuracy_history.append(train_accuracy)

    # Log training progress
    print(f'Epoch [{epoch+1}/{config.epochs}], Loss: {average_train_loss:.4f}, F1: {train_f1:.4f}, Accuracy: {train_accuracy:.4f}')

    # Add validation and early stopping logic here if necessary
#endregion

tensor(0, device='mps:0')
tensor(-3.6962, device='mps:0') tensor(4.2990, device='mps:0')
tensor(0, device='mps:0')
tensor(0., device='mps:0', grad_fn=<MinBackward1>) tensor(0., device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(-10.3720, device='mps:0', grad_fn=<MinBackward1>) tensor(8.0534, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(-10.3827, device='mps:0', grad_fn=<MinBackward1>) tensor(13.5245, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(0., device='mps:0', grad_fn=<MinBackward1>) tensor(0., device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(-10.0435, device='mps:0', grad_fn=<MinBackward1>) tensor(20.0259, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(-1.5289e+34, device='mps:0', grad_fn=<MinBackward1>) tensor(2.0099e+36, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0, device='mps:0')
tensor(-9.1698e+33, device='mps:0', grad_fn=<MinBackward1>) tensor(1

In [ ]:
    # ------------------------------------------------------------------------------------------------------- #

    # - VAL
    total_val_loss, total_val_mse_loss, total_val_similarity_loss, total_val_activity_loss = 0, 0, 0, 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    pose2imu_model.eval()
    fe_model.eval()
    ac_model.eval()
    with torch.no_grad():
        for accel, skel, label in mhad_dataloader:
            # -- Forward pass
            # --- Regressor
            sim_accel = pose2imu_model(skel)
            mse_loss = MSELoss(sim_accel, accel)
            total_val_mse_loss += mse_loss.item()
            # --- Feature Extractor
            accel_f = fe_model(accel)
            sim_accel_f = fe_model(sim_accel)
            similarity_loss = cosine_similarity_loss(
                sim_accel_f, accel_f)
            total_val_similarity_loss += similarity_loss.item()
            # --- Activity Classifier
            label_logits = ac_model(accel_f)
            sim_label_logits = ac_model(sim_accel_f)
            activity_loss = CrossEntropyLoss(
                label_logits, label) + CrossEntropyLoss(sim_label_logits, label)
            total_val_activity_loss += activity_loss.item()
            # -- Total Loss
            total_loss = mse_loss + config.scenario2_alpha * \
                activity_loss + config.scenario2_beta * similarity_loss
            total_val_loss += total_loss.item()

            # -- F1
            pred_labels = torch.argmax(label_logits, dim=1)
            all_pred_labels.extend(pred_labels.cpu().numpy())
            all_true_labels.extend(label.cpu().numpy())

            # -- Accuracy
            total_predictions += label.size(0)
            correct_predictions += (pred_labels == label).sum().item()

    # -- Total Loss
    average_val_loss = total_val_loss / len(mmfit.val_loader)
    val_loss_history.append(average_val_loss)
    # -- MSE Loss
    average_val_mse_loss = total_val_mse_loss / len(mmfit.val_loader)
    val_mse_loss_history.append(average_val_mse_loss)
    # -- Similarity Loss
    average_val_similarity_loss = total_val_similarity_loss / \
        len(mmfit.val_loader)
    val_similarity_loss_history.append(average_val_similarity_loss)
    # -- Activity Loss
    average_val_activity_loss = total_val_activity_loss / len(mmfit.val_loader)
    val_activity_loss_history.append(average_val_activity_loss)

    # -- F1
    val_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    val_f1_history.append(val_f1)

    # -- Accuracy
    val_accuracy = correct_predictions / total_predictions
    val_accuracy_history.append(val_accuracy)

    # ------------------------------------------------------------------------------------------------------- #

    out = (f"Epoch {epoch+1}/{epochs}, alpha: {config.scenario2_alpha}, beta: {config.scenario2_beta}" +
           f"\nTRAIN Total Loss: {average_train_loss:.4f}, MSE Loss: {average_train_mse_loss:.4f}, Activity Loss * alpha: {average_train_activity_loss * config.scenario2_alpha:.4f}, Similarity Loss * beta: {average_train_similarity_loss * config.scenario2_beta:.4f}" +
           f'\nTRAIN F1: {train_f1:.4f}, Accuracy: {train_accuracy:.4f}' +
           f'\nVAL Total Loss: {average_val_loss:.4f}, MSE Loss: {average_val_mse_loss:.4f}, Activity Loss * alpha: {average_val_activity_loss * config.scenario2_alpha:.4f}, Similarity Loss * beta: {average_val_similarity_loss * config.scenario2_beta:.4f}' +
           f'\nVAL F1: {val_f1:.4f}, Accuracy: {val_accuracy:.4f}' +
           f'\n----------------------------------------------------\n')

    print(out)

    if best_val_f1 < val_f1:
        epochs_no_improve = 0

        best_val_f1 = val_f1
        best_val_acc = val_accuracy
        best_epoch = epoch

        best_pose2imu_model_state = copy.deepcopy(pose2imu_model.state_dict())
        best_fe_model_state = copy.deepcopy(fe_model.state_dict())
        best_ac_model_state = copy.deepcopy(ac_model.state_dict())

        log = out

    else:
        epochs_no_improve += 1

    if epochs_no_improve == patience:
        pose2imu_model.load_state_dict(best_pose2imu_model_state)
        fe_model.load_state_dict(best_fe_model_state)
        ac_model.load_state_dict(best_ac_model_state)

        break

    scheduler.step(average_val_loss)

In [ ]:
# >>> Training <<< #
train_loss_history, train_f1_history, train_accuracy_history = [], [], []
val_loss_history, val_f1_history, val_accuracy_history = [], [], []

best_fe_model_state, best_ac_model_state = None, None
best_epoch = float('inf')
best_val_f1 = 0
best_val_acc = 0

epochs = config.epochs
patience = config.patience
epochs_no_improve = 0

log = ''

for epoch in range(epochs):

    # - TRAIN
    total_train_loss = 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    fe_model.train()
    ac_model.train()
    for skel, accel, label in mmfit.train_loader:
        """
        pose: (batch_size, 3, num_joints, sensor_window_length)
        acc: (batch_size, 3, sensor_window_length)
        """
        # -- Move to GPU
        skel = skel.to(config.device, non_blocking=True)
        accel = accel.to(config.device, non_blocking=True)
        label = label.to(config.device, non_blocking=True)

        # -- Forward pass
        # --- Regressor
        if config.mode == config.Mode.SIM:
            accel = pose2imu_model(skel)  # IMPORTANT SIM acc
        # --- Feature Extractor
        acc_features = fe_model(accel)
        # --- Activity Classifier
        label_logits = ac_model(acc_features)

        # -- Loss
        loss = criterion(label_logits, label)
        total_train_loss += loss.item()

        # -- F1
        pred_labels = torch.argmax(label_logits, dim=1)
        all_pred_labels.extend(pred_labels.cpu().numpy())
        all_true_labels.extend(label.cpu().numpy())

        # -- Accuracy
        total_predictions += label.size(0)
        correct_predictions += (pred_labels == label).sum().item()

        # -- Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # -- Loss
    average_train_loss = total_train_loss / len(mmfit.train_loader)
    train_loss_history.append(average_train_loss)

    # -- F1
    train_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    train_f1_history.append(train_f1)

    # -- Accuracy
    train_accuracy = correct_predictions / total_predictions
    train_accuracy_history.append(train_accuracy)

    # ------------------------------------------------------------------------------------------------------- #

    # - VAL
    total_val_loss = 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    fe_model.eval()
    ac_model.eval()
    with torch.no_grad():
        for skel, accel, label in mmfit.val_loader:
            # -- Move to GPU
            skel = skel.to(config.device, non_blocking=True)
            accel = accel.to(config.device, non_blocking=True)
            label = label.to(config.device, non_blocking=True)

            # -- Forward pass
            # --- Feature Extractor
            acc_features = fe_model(accel)
            # --- Activity Classifier
            label_logits = ac_model(acc_features)

            # -- Loss
            loss = criterion(label_logits, label)
            total_val_loss += loss.item()

            # -- F1
            pred_labels = torch.argmax(label_logits, dim=1)
            all_pred_labels.extend(pred_labels.cpu().numpy())
            all_true_labels.extend(label.cpu().numpy())

            # -- Accuracy
            total_predictions += label.size(0)
            correct_predictions += (pred_labels == label).sum().item()

    # -- Loss
    average_val_loss = total_val_loss / len(mmfit.val_loader)
    val_loss_history.append(average_val_loss)

    # -- F1
    val_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    val_f1_history.append(val_f1)

    # -- Accuracy
    val_accuracy = correct_predictions / total_predictions
    val_accuracy_history.append(val_accuracy)

    # ------------------------------------------------------------------------------------------------------- #

    out = (f"Epoch {epoch+1}/{epochs}" +
           f"\nTRAIN Total Loss: {average_train_loss:.4f}" +
           f'\nTRAIN F1: {train_f1:.4f}, Accuracy: {train_accuracy:.4f}' +
           f'\nVAL Total Loss: {average_val_loss:.4f}' +
           f'\nVAL F1: {val_f1:.4f}, Accuracy: {val_accuracy:.4f}' +
           f'\n----------------------------------------------------\n')

    print(out)

    # - Early stop
    if best_val_f1 < val_f1:
        epochs_no_improve = 0

        best_val_f1 = val_f1
        best_val_acc = val_accuracy
        best_epoch = epoch
        
        best_fe_model_state = copy.deepcopy(fe_model.state_dict())
        best_ac_model_state = copy.deepcopy(ac_model.state_dict())
        
        log = out

    else:
        epochs_no_improve += 1

    # Training early stop
    if epochs_no_improve == patience:
        # -- Load best model
        fe_model.load_state_dict(best_fe_model_state)
        ac_model.load_state_dict(best_ac_model_state)

        break

    scheduler.step(average_val_loss)

In [ ]:
# >>> NTU data (fake) <<<
skel_data = []

pose_dir = config.ntu_data_dir
folders = sorted([f for f in os.listdir(pose_dir)
                 if os.path.isdir(os.path.join(pose_dir, f))])

for folder in tqdm(folders, desc='folders'):
    files = sorted([f for f in os.listdir(os.path.join(pose_dir, folder))])

    for file in tqdm(files, desc='files'):
        if file.endswith('norm.npy'):
            # shape (3, N, 17)
            skel = np.load(os.path.join(pose_dir, folder, file))
            skel_data.append(skel[:, :, [11, 12, 13]]) # only select left arm

ntu_dataset = ntu_data.NTUDataset(data=skel_data)
ntu_dataloader = DataLoader(ntu_dataset, batch_size=config.batch_size, shuffle=True, num_workers=0)

In [4]:
# >>> MHAD accelerometer data (real) <<< #
all_acc = []
all_labels = []
for s in config.MHAD_TRAIN_S_IDS:
    s_dir = os.path.join(config.mhad_data_dir, s)

    files = sorted(os.listdir(s_dir))
    # group modalities
    for file in files:
        if file.endswith(config.mhad_inertial_file):
            accel = np.load(os.path.join(s_dir, file))
            all_acc.append(accel)
            label = file.split('_')[0]
            all_labels.append(label)

In [13]:
pose2imu_model = Regressor(
    in_ch=config.in_ch,
    num_joints=config.num_joints,
    window_length=config.sensor_window_length
).to(config.device)
best_seed = config.best_pose2imu_seed # best_seed of best model
model_name = config.pose2imu_model_name + f"[{best_seed}](model).pth"
latest_model = utils.find_latest_model(model_name)
pose2imu_model.load_state_dict(torch.load(latest_model, map_location=config.device))
pose2imu_model.eval() # we're not training this model

Regressor(
  (tcn1): TCNBlock(
    (conv0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (conv1): Conv2d(3, 32, kernel_size=(3, 5), stride=(1, 1), padding=same)
    (dropout1): Dropout(p=0, inplace=False)
    (conv2): Conv2d(32, 32, kernel_size=(3, 5), stride=(1, 1), padding=same)
    (dropout2): Dropout(p=0, inplace=False)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (tcn2): TCNBlock(
    (conv0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (conv1): Conv2d(32, 32, kernel_size=(3, 5), stride=(1, 1), padding=same, dilation=(2, 2))
    (dropout1): Dropout(p=0.2, inplace=False)
    (conv2): Conv2d(32, 32, kernel_size=(3, 5), stride=(1, 1), padding=same, dilation=(2, 2))
    (dropout2): Dropout(p=0.2, inplace=False)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (tcn3): TCNBlock(
    (conv0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (conv1): Conv2d(32, 32, kernel_size=(3, 5), stride=(1, 1), padding=sam

In [14]:
# >>> Real Fake Dataset <<<
# MM-Fit data (real)
data_dir = config.mmfit_data_dir
real_acc = []
for w_id in config.EXPERIMENT_W_IDS:
    id_dir = os.path.join(data_dir, w_id)
    acc_file = os.path.join(id_dir, w_id + "_" + config.acc_file) 
    accel = np.load(acc_file) # (N, 5)
    real_acc.append(accel[:, 2:].transpose(1, 0)) # (N, 3) 

In [15]:
# Fake acc from NTU data
fake_acc = []
with torch.no_grad():
    for skel, _ in ntu_dataloader:
        pred = pose2imu_model(skel)
        pred = pred.cpu().numpy()
        for i in range(pred.shape[0]):
            fake_acc.append(pred[i])

In [16]:
real_fake_dataset = real_fake_data.RealFakeDataset(real=real_acc, fake=fake_acc)
real_fake_dataloader = DataLoader(real_fake_dataset, batch_size=config.batch_size, shuffle=True, num_workers=0)

88.61343326607091% of data is real
11.386566733929087% of data is fake


In [33]:
# >>> Models <<< #
discriminator = Discriminator(input_size=900).to(config.device)
fe_model = FeatureExtractor().to(config.device, non_blocking=True)
ac_model = ActivityClassifier(f_in=config.ac_fin, n_classes=config.ac_num_classes).to(config.device, non_blocking=True)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(discriminator.parameters(), lr=config.lr)

In [38]:
epochs = 100
for epoch in range(epochs):
    for real_fake, label in real_fake_dataloader:
        optimizer.zero_grad()
        outputs = discriminator(real_fake)
        loss = criterion(outputs.squeeze(), label)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 1/100, Loss: 0.0622
Epoch 2/100, Loss: 0.0319
Epoch 3/100, Loss: 0.0362
Epoch 4/100, Loss: 0.0049
Epoch 5/100, Loss: 0.0057
Epoch 6/100, Loss: 0.0023
Epoch 7/100, Loss: 0.0010
Epoch 8/100, Loss: 0.0020
Epoch 9/100, Loss: 0.0009
Epoch 10/100, Loss: 0.0005
Epoch 11/100, Loss: 0.0005
Epoch 12/100, Loss: 0.0003
Epoch 13/100, Loss: 0.0004
Epoch 14/100, Loss: 0.0008
Epoch 15/100, Loss: 0.0001
Epoch 16/100, Loss: 0.0003
Epoch 17/100, Loss: 0.0003
Epoch 18/100, Loss: 0.0001
Epoch 19/100, Loss: 0.0001
Epoch 20/100, Loss: 0.0002
Epoch 21/100, Loss: 0.0002
Epoch 22/100, Loss: 0.0002
Epoch 23/100, Loss: 0.0002
Epoch 24/100, Loss: 0.0001


KeyboardInterrupt: 

In [14]:
for data, label in real_fake_dataloader:
    if not torch.all(label==0.0) and not torch.all(label == 1.0):
        print(label)
        break


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0')
